In [2]:
!du -sh

du: cannot read directory ‘./lost+found’: Permission denied
13G	.


In [3]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 3.6 MB/s eta 0:00:0000:01


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [5]:
!pip install datasets -q seqeval

In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
dataset = load_dataset('SotiriosKastanas/try_new')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1020 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/SotiriosKastanas___parquet/nielsr--funsd-image-feature-3d0946c298829077/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 100
    })
})

In [10]:
from torch.utils.data import Dataset
from PIL import Image
import torch

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

class CustomDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    # get item
    example = self.dataset[idx]
    image = example["image"]
    words = example["tokens"]
    boxes = example["bboxes"]
    ner_tags = example["ner_tags"]

    # prepare for the model
    width, height = image.size

    bbox = []
    labels = []
    for word, box, label in zip(words, boxes, ner_tags):
        box = normalize_bbox(box, width, height)
        n_word_tokens = len(tokenizer.tokenize(word))
        bbox.extend([box] * n_word_tokens)
        labels.extend([label] + ([-100] * (n_word_tokens - 1)))

    cls_box = sep_box = [0, 0, 0, 0]
    bbox = [cls_box] + bbox + [sep_box]
    labels = [-100] + labels + [-100]

    encoding = self.tokenizer(" ".join(words), truncation=True, max_length=512)
    sequence_length = len(encoding.input_ids)
    # truncate boxes and labels based on length of input ids
    labels = labels[:sequence_length]
    bbox = bbox[:sequence_length] 

    encoding["bbox"] = bbox
    encoding["labels"] = labels
    
    return encoding

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nielsr/lilt-xlm-roberta-base")

In [12]:
train_dataset = CustomDataset(dataset["train"], tokenizer)
eval_dataset = CustomDataset(dataset["test"], tokenizer)

In [14]:
from torch.utils.data import DataLoader

def collate_fn(features):
  boxes = [feature["bbox"] for feature in features]
  labels = [feature["labels"] for feature in features]
  # use tokenizer to pad input_ids
  batch = tokenizer.pad(features, padding="max_length", max_length=512)

  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
  batch["labels"] = [labels_example + [-100] * (sequence_length - len(labels_example)) for labels_example in labels]
  batch["bbox"] = [boxes_example + [[0, 0, 0, 0]] * (sequence_length - len(boxes_example)) for boxes_example in boxes]

  # convert to PyTorch
  # batch = {k: torch.tensor(v, dtype=torch.int64) if isinstance(v[0], list) else v for k, v in batch.items()}
  batch = {k: torch.tensor(v) for k, v in batch.items()}

  return batch

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [15]:
batch = next(iter(train_dataloader))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
label_list = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']
id2label = {id:label for id, label in enumerate(label_list)}
print(id2label)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22'}


In [17]:
from transformers import LiltForTokenClassification
model = LiltForTokenClassification.from_pretrained("nielsr/lilt-xlm-roberta-base", id2label=id2label)

Some weights of LiltForTokenClassification were not initialized from the model checkpoint at nielsr/lilt-xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import evaluate
metric = evaluate.load("seqeval")     

In [19]:
import numpy as np
from seqeval.metrics import classification_report

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="FINAL_ENGLISH",
                                  num_train_epochs=10,
                                  learning_rate=5e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:

from transformers.data.data_collator import default_data_collator

class CustomTrainer(Trainer):
  def get_train_dataloader(self):
    return train_dataloader

  def get_eval_dataloader(self, eval_dataset = None):
    return eval_dataloader

# Initialize our Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [22]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [23]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.573061,0.159218,0.245690,0.193220,0.856497
200,No log,0.754801,0.185864,0.306034,0.231270,0.850635
300,No log,0.709396,0.333333,0.327586,0.330435,0.887470
400,No log,0.775571,0.336000,0.362069,0.348548,0.897275
500,0.583500,0.736545,0.456522,0.543103,0.496063,0.910338
600,0.583500,0.639128,0.434641,0.573276,0.494424,0.935015
700,0.583500,0.647260,0.522727,0.594828,0.556452,0.919673
800,0.583500,0.635220,0.322957,0.357759,0.339468,0.905634
900,0.583500,0.587097,0.475694,0.590517,0.526923,0.926222
1000,0.271700,0.541353,0.542435,0.633621,0.584493,0.938452


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

TrainOutput(global_step=5100, training_loss=0.16778044803469788, metrics={'train_runtime': 1597.1257, 'train_samples_per_second': 6.386, 'train_steps_per_second': 3.193, 'total_flos': 2837645990092800.0, 'train_loss': 0.16778044803469788, 'epoch': 10.0})

In [25]:
train_dataset[0]

{'input_ids': [0, 159, 75930, 9, 841, 22429, 2452, 40266, 47, 42477, 219790, 53, 119210, 8357, 98872, 36404, 24241, 11338, 6, 114999, 130997, 4, 190, 37967, 54, 14, 15110, 5, 2839, 11540, 34841, 64, 114999, 130997, 42477, 24241, 581, 189940, 13, 357, 1121, 329, 674, 111, 1061, 28483, 239, 474, 8613, 9763, 3584, 8509, 19888, 23, 197527, 7, 930, 1052, 4021, 5024, 449, 4, 418, 41115, 95950, 66, 567, 1132, 4, 304, 136, 108571, 14, 441, 8467, 7550, 418, 106, 29693, 111, 219790, 53, 119210, 4, 4954, 119543, 12535, 4, 4954, 119543, 57947, 167713, 4, 9098, 116, 63557, 111, 130480, 2408, 4, 3291, 1463, 1159, 82821, 20387, 8262, 1294, 136, 219790, 53, 29693, 4, 6098, 119543, 57947, 7739, 4, 9098, 46348, 18908, 555, 329, 5608, 186, 29823, 297, 47, 930, 1052, 4021, 5024, 449, 4, 1647, 7, 32189, 238, 7569, 981, 43741, 5, 277, 5, 27765, 93908, 14, 4126, 1438, 8055, 1877, 74, 63441, 5281, 209, 18982, 1532, 74, 111383, 297, 116, 11994, 1532, 198919, 69340, 12, 204990, 96560, 2529, 4189, 950, 1532, 930

In [35]:
dataset['train']['tokens'][0]

['SAGE-Hindawi',
 'Access',
 'to',
 'Research',
 'Veterinary',
 'Medicine',
 'International',
 'Volume',
 '2010,',
 'Article',
 'ID',
 '485060,',
 '5',
 'pages',
 'doi:10.4061/2010/485060',
 'Research',
 'Article',
 'The',
 'Immune',
 'Enhancement',
 'of',
 'Sodium',
 'Lauryl',
 'Sulfoacetate',
 'in',
 'Chickens',
 'DaRong',
 'Cheng,1',
 'ShanYuan',
 'Zhu,2',
 'and',
 'HuaiChang',
 'Sun1',
 '1',
 'College',
 'of',
 'Veterinary',
 'Medicine,',
 'Yangzhou',
 'University,',
 'Yangzhou',
 '225009,',
 'China',
 '2',
 'Department',
 'of',
 'Pharmacy,',
 'Jiangsu',
 'Animal',
 'Husbandry',
 'and',
 'Veterinary',
 'College,',
 'Taizhou',
 '225300,',
 'China',
 'Correspondence',
 'should',
 'be',
 'addressed',
 'to',
 'DaRong',
 'Cheng,',
 'jsyzcdr@yahoo.com.cn',
 'Received',
 '27',
 'August',
 '2009;',
 'Revised',
 '10',
 'January',
 '2010;',
 'Accepted',
 '2',
 'March',
 '2010',
 'Academic',
 'Editor:',
 'Maurice',
 'Pensaert',
 'Copyright',
 '©',
 '2010',
 'DaRong',
 'Cheng',
 'et',
 'al.',


In [32]:
train_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'bbox', 'labels'])

In [34]:
len(train_dataset[0]['input_ids'])

512

In [1]:
#remove the list

In [7]:
from datasets import load_dataset
dataset = load_dataset('SotiriosKastanas/chinese_try')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1020 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/SotiriosKastanas___parquet/nielsr--funsd-image-feature-214b37971ee5c79e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 1020
    })
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 100
    })
})

In [9]:
from torch.utils.data import Dataset
from PIL import Image
import torch

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

class CustomDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    # get item
    example = self.dataset[idx]
    image = example["image"]
    words = example["tokens"]
    boxes = example["bboxes"]
    ner_tags = example["ner_tags"]

    # prepare for the model
    width, height = image.size

    bbox = []
    labels = []
    for word, box, label in zip(words, boxes, ner_tags):
        box = normalize_bbox(box, width, height)
        n_word_tokens = len(tokenizer.tokenize(word))
        bbox.extend([box] * n_word_tokens)
        labels.extend([label] + ([-100] * (n_word_tokens - 1)))

    cls_box = sep_box = [0, 0, 0, 0]
    bbox = [cls_box] + bbox + [sep_box]
    labels = [-100] + labels + [-100]

    encoding = self.tokenizer(" ".join(words), truncation=True, max_length=512)
    sequence_length = len(encoding.input_ids)
    # truncate boxes and labels based on length of input ids
    labels = labels[:sequence_length]
    bbox = bbox[:sequence_length] 

    encoding["bbox"] = bbox
    encoding["labels"] = labels
    
    return encoding

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nielsr/lilt-xlm-roberta-base")

In [11]:
train_dataset = CustomDataset(dataset["train"], tokenizer)
eval_dataset = CustomDataset(dataset["test"], tokenizer)

In [12]:
from torch.utils.data import DataLoader

def collate_fn(features):
  boxes = [feature["bbox"] for feature in features]
  labels = [feature["labels"] for feature in features]
  # use tokenizer to pad input_ids
  batch = tokenizer.pad(features, padding="max_length", max_length=512)

  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
  batch["labels"] = [labels_example + [-100] * (sequence_length - len(labels_example)) for labels_example in labels]
  batch["bbox"] = [boxes_example + [[0, 0, 0, 0]] * (sequence_length - len(boxes_example)) for boxes_example in boxes]

  # convert to PyTorch
  # batch = {k: torch.tensor(v, dtype=torch.int64) if isinstance(v[0], list) else v for k, v in batch.items()}
  batch = {k: torch.tensor(v) for k, v in batch.items()}

  return batch

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [13]:
batch = next(iter(train_dataloader))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [14]:
label_list = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']
id2label = {id:label for id, label in enumerate(label_list)}
print(id2label)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22'}


In [15]:
from transformers import LiltForTokenClassification
model = LiltForTokenClassification.from_pretrained("nielsr/lilt-xlm-roberta-base", id2label=id2label)

Some weights of LiltForTokenClassification were not initialized from the model checkpoint at nielsr/lilt-xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import evaluate
metric = evaluate.load("seqeval")     

In [17]:
import numpy as np
from seqeval.metrics import classification_report

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="FINAL_CHINESE",
                                  num_train_epochs=10,
                                  learning_rate=5e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:

from transformers.data.data_collator import default_data_collator

class CustomTrainer(Trainer):
  def get_train_dataloader(self):
    return train_dataloader

  def get_eval_dataloader(self, eval_dataset = None):
    return eval_dataloader

# Initialize our Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [20]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [21]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.833296,0.040201,0.038095,0.039120,0.758039
200,No log,1.033639,0.022727,0.042857,0.029703,0.721538
300,No log,0.851099,0.065657,0.061905,0.063725,0.710393
400,No log,0.920149,0.091153,0.161905,0.116638,0.760135
500,0.502600,0.771201,0.181818,0.238095,0.206186,0.778743
600,0.502600,0.587870,0.145390,0.195238,0.166667,0.854711
700,0.502600,0.742778,0.242424,0.304762,0.270042,0.836972
800,0.502600,0.847369,0.223729,0.314286,0.261386,0.778334
900,0.502600,0.659108,0.105121,0.185714,0.134251,0.791933
1000,0.249600,0.741885,0.177358,0.223810,0.197895,0.825520


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 6 seems not to be NE tag.
  wa

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 20 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 8 seems not to be NE tag.
  w

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 16 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ec2-user/anaconda3/envs/pytorch_p

TrainOutput(global_step=5100, training_loss=0.1515537649042466, metrics={'train_runtime': 1625.6513, 'train_samples_per_second': 6.274, 'train_steps_per_second': 3.137, 'total_flos': 2837645990092800.0, 'train_loss': 0.1515537649042466, 'epoch': 10.0})